In [5]:
def f_alpha(alpha, fun, x, s, args=()) :
    """
    This is a one-dimensional version of the objective function
    given by the parameter alpha
    
    alpha : 1D independent variable
    fun   : Original objective function
    x     : Start point
    s     : 1D search direction
    args  : Tuple extra arguments passed to the objective function
    """
    x_new = x + alpha * s
    
    return fun(x_new, *args)

def gss(fun, dfun, x, s, args=(), delta=1.0e-2, tol=1e-15):
    '''
    Line search function by golden section search
    https://en.wikipedia.org/wiki/Golden-section_search and [arora]
    
    fun   : Original objective function
    dfun  : Objective function gradient which is not used
    x     : Start point
    s     : 1D search directin
    args  : Tuple extra arguments passed to the objective function
    delta : Init. guess interval determining initial interval of uncertainty
    tol   : stop criterion
    '''
    gr = (np.sqrt(5) + 1) / 2
    
    ########################################################################################
    # ESTABLISH INITIAL DELTA
    # 초기 delta를 잡는다.
    # alpah = 0에서 값과 delta에서의 함수값을 계산하고 delta에서의 값이 크다면 delta를 줄인다.
    ########################################################################################
    AL = 0.
    FL = f_alpha(AL, fun, x, s, args)
    AA = delta
    FA = f_alpha(AA, fun, x, s, args)
    while  FL < FA :
        delta = 0.1*delta
        AA = delta
        FA = f_alpha(AA, fun, x, s, args)
    ########################################################################################
    
    ########################################################################################
    # ESTABLISH INITIAL INTERVAL OF UNCERTAINTY
    # delta를 사용하여 초기 불확정 구간을 설정한다.
    # 결정된 구간을 [AL, AU] 로 두고 황금분할 탐색을 시작한다.
    ########################################################################################
    j = 1
    AU = AA + delta * (gr**j)
    FU = f_alpha(AU, fun, x, s, args)
    while FA > FU :
        AL = AA
        AA = AU
        FL = FA
        FA = FU
        
        j += 1
        AU = AA + delta * (gr**j)
        FU = f_alpha(AU, fun, x, s, args)

    AB = AL + (AU - AL) / gr
    FB = f_alpha(AB, fun, x, s, args)
    
    while abs(AA - AB) > tol:
        if f_alpha(AA, fun, x, s, args) < f_alpha(AB, fun, x, s, args):
            AU = AB
        else:
            AL = AA

        # we recompute both c and d here to avoid loss of precision which may lead to incorrect results or infinite loop
        AA = AU - (AU - AL) / gr
        AB = AL + (AU - AL) / gr

    return ( (AU + AL) / 2, )


def numer_grad(x, fun, args, dtype='float64'):
    """
    Find the first derivative of a function at a point x.
    
    x     : The point at which derivative is found.
    fun   : Input function.
    args  : Tuple extra arguments passed to fun
    dtype : Data type of gradient, must be set to 'longdouble' if numerically unstable 
    """
    g = np.zeros(x.shape[0]).astype(dtype)
    
    for i in range(x.shape[0]) :
        dx1 = x.copy()
        dx2 = x.copy()
        
        # h 결정 대충 변수의 1%정도
        # https://en.wikipedia.org/wiki/Numerical_differentiation
        h = np.sqrt(np.finfo(np.float64).eps) * x[i]
        # xph = w[i] + h
        # h = xph - w[i]
        
        ##########################################################
        # 식으로는 동일하나 아래쪽으로 더 많이 구현됨
        # dw1[i] += h/2.
        # dw2[i] -= h/2.
        # g[i] = (E(dw1, M, x, t) - E(dw2, M, x, t)) / h
        
        dx1[i] += h
        dx2[i] -= h
        g[i] = ( fun(dx1, *args) - fun(dx2, *args) ) / (2*h)
        
    return g


def minimize(fun, x0, args=(), method='CGFR', jac=None, linesrch=gss, lineargs={},
             max_iter=2500, eps=1.0e-7, strict=True, callback=None, 
             verbose=True, verbose_step=10, debug=False) :
    """
    Minimize the given function, fun.
    
    fun          : The objective function to be minimized. fun(x, *args) -> float
    x0           : Initial guess.
    args         : Tuple extra arguments passed to the objective function
    method       : 'CGFR'    : Fletcher-Reeves conjugate gradient
                   'CGPR'    : Polak and Ribiere
                   Otherwise : steepest descent
    jac          : Method for computing the gradient vector
    linesrch     : Line search(1D optimize) function, default : golden section search(gss)
    lineargs     : Tuple extra arguments passed to the line search function
    max-iter     : Maximum number of iterations to perform.
    eps          : Stop criterion
    strict       : True : Check if the objective function is always decreasing
    verbose      : Print process info.
    verbose_step : Set how often to print process info.
    debug        : Print debug info.
    """
    
    methods = {'CGFR':'Conjugate gradient Fletcher-Reeves', 'CGPR':'Conjugate gradient Polak and Ribiere'}
    
    #################################################################
    # initialize
    #################################################################
#     np.random.seed(0)
#     if x0 is None :
#         x = np.random.uniform(-1, 1, M+1)
#     else :
    x = x0
        
    ################################################################
    # information
    #################################################################
    if verbose == True :
        print("################################################################")
        print("# START OPTIMIZATION")
        print("################################################################")
        print("INIT POINT : {}, dtype : {}".format(x, x.dtype))
        print("METHOD     : {}".format(methods[method]  if method in ["CGFR","CGPR"] else "Steepest Descent"))
        print("##############")
        print("# START ITER.")
        print("##############")

    for i in range(max_iter): # while True :
        # calc. gradient
        if jac == None :
            c = numer_grad(x, fun, args)
        else :
            c = jac(x, *args)

        if debug == True :
            print("{}th gradient {}".format(i, c))
            
        # stop criterion
        if np.linalg.norm(c) < eps :
            if verbose==True :
                print('Stop criterion break')
                print( "Iter:{:5d}, |c|:{:11.7f}, alpha:{:.7f}, Cost:{:11.7f}, d:{}, x:{}".format(i+1, np.linalg.norm(c), 
                                                                                       alpha, fun(x, *args), d, x) )
            break

        # make descent direction
        if i > 0 and method == "CGFR" :   
            # Fletcher-Reeves (original method)
            beta = (np.linalg.norm(c) / np.linalg.norm(c_old))**2
            d = -c + beta*d_old
        elif i > 0 and method == "CGPR" :   
            #Polak and Ribiere
            beta = (np.dot(c, c-c_old)) / np.linalg.norm(c_old)**2
            d = -c + beta*d_old
        else :
            d = -c # for steepest descent method 

        if debug == True :
            print("{}th descent direction {}".format(i+1, d))

        # line search 
        if linesrch != None :
            alpha = linesrch(fun, jac, x, d, args=args, **lineargs)[0]
        else :
            alpha = lineargs["lr"]
        
        if debug == True :
            print("{}th alpha {}".format(i+1, alpha))
            
        # Update design variables
        cost_old = fun(x, *args)
        
        if debug == True :
            print("{}th before update x {}".format(i+1, x))
        
        if alpha == None :
            print("line search fail")
            break
        else :
            x = x + alpha * d
        
        if debug == True :
            print("{}th after updata x {}".format(i+1, x))
        
        cost_new = fun(x, *args)
        c_old = c.copy()
        d_old = d.copy()

        # Check for increasing the objective function.
        # 강하조건을 만족시키지 않을 수 있는 경우 이 조건을 끌 수 있게 해야 한다.
        if strict == True and cost_new > cost_old :
            if verbose == True :
                print("Numerical unstable break : iter:{:5d}, Cost_old:{:.7f}, Cost_new:{:.7f}".format(i+1, cost_old, cost_new))
            break

        # print information.
        if verbose==True and i % verbose_step == 0 :
            print( "Iter:{:5d}, |c|:{:11.7f}, alpha:{:.7f}, Cost:{:11.7f}, d:{}, x:{}".format(i+1, np.linalg.norm(c), 
                                                                                       alpha, fun(x, *args), d, x) )

        if callback :
            callback(x)    
    else :    
        if verbose==True :
            print('max-iter break')
            print( "Iter:{:5d}, |c|:{:11.7f}, alpha:{:.7f}, Cost:{:11.7f}, d:{}, x:{}".format(i+1, np.linalg.norm(c), 
                                                                                       alpha, fun(x, *args), d, x) )
        
    return x

In [1]:
%%html
<link href='https://fonts.googleapis.com/earlyaccess/notosanskr.css' rel='stylesheet' type='text/css'>
<!--https://github.com/kattergil/NotoSerifKR-Web/stargazers-->
<link href='https://cdn.rawgit.com/kattergil/NotoSerifKR-Web/5e08423b/stylesheet/NotoSerif-Web.css' rel='stylesheet' type='text/css'>
<!--https://github.com/Joungkyun/font-d2coding-->
<link href="http://cdn.jsdelivr.net/gh/joungkyun/font-d2coding/d2coding.css" rel="stylesheet" type="text/css">
<style>
    h1     { font-family: 'Noto Sans KR' !important; color:#348ABD !important;   }
    h2     { font-family: 'Noto Sans KR' !important; color:#467821 !important;   }
    h3, h4 { font-family: 'Noto Sans KR' !important; color:#A60628 !important;   }
    p:not(.navbar-text) { font-family: 'Noto Serif KR', 'Nanum Myeongjo'; font-size: 12pt; line-height: 200%;  text-indent: 10px; }
    li:not(.dropdown):not(.p-TabBar-tab):not(.p-MenuBar-item):not(.jp-DirListing-item):not(.p-CommandPalette-header):not(.p-CommandPalette-item):not(.jp-RunningSessions-item)
            { font-family: 'Noto Serif KR', 'Nanum Myeongjo'; font-size: 12pt; line-height: 200%; }
    table  { font-family: 'Noto Sans KR' !important;  font-size: 11pt !important; }           
    li > p  { text-indent: 0px; }
    li > ul { margin-top: 0px !important; }       
    sup { font-family: 'Noto Sans KR'; font-size: 9pt; } 
    code, pre  { font-family: D2Coding, 'D2 coding' !important; font-size: 12pt !important; line-height: 130% !important;}
    .code-body { font-family: D2Coding, 'D2 coding' !important; font-size: 12pt !important;}
    .ns        { font-family: 'Noto Sans KR'; font-size: 15pt;}
    .summary   {
                   font-family: 'Georgia'; font-size: 12pt; line-height: 200%; 
                   border-left:3px solid #FF0000; 
                   padding-left:20px; 
                   margin-top:10px;
                   margin-left:15px;
               }
    .green { color:#467821 !important; }
    .comment { font-family: 'Noto Sans KR'; font-size: 10pt; }
</style>